In [ ]:
import pandas as pd
import numpy as np

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from cataclop.ml import preprocessing
from cataclop.ml import exploration

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

from cataclop.ml.pipeline import factories

In [4]:
dataset = factories.Dataset.factory('default', params={
    'from': '2020-01-01',
    'to': '2020-02-01',
})
dataset.load()

1152 races


In [5]:
df = dataset.players
df.describe()

,race_id,age,num,race_count,victory_count,placed_count,placed_2_count,placed_3_count,earnings,victory_earnings,placed_earnings,year_earnings,prev_year_earnings,post_position,position,handicap_weight,handicap_distance,trueskill_mu,trueskill_sigma,time,hist_1_days,hist_2_days,hist_3_days,winner_dividend,placed_dividend,final_odds,final_odds_offline,final_odds_unibet,final_odds_ref,final_odds_ref_offline,final_odds_ref_unibet,trainer_winning_rate,trainer_avg_winning_dividend,horse_id,trainer_id,jockey_id,herder_id,owner_id,num_race,num_bis,prize,distance,declared_player_count,session_id,num_session,hippodrome_id,hist_1_pos,hist_2_pos,hist_3_pos,hist_4_pos,hist_5_pos,hist_6_pos,speed,win,final_odds_ref_inv,race_count_mean,race_count_std,race_count_amin,race_count_amax,hist_1_pos_mean,hist_1_pos_std,hist_1_pos_amin,hist_1_pos_amax,hist_2_pos_mean,hist_2_pos_std,hist_2_pos_amin,hist_2_pos_amax,hist_3_pos_mean,hist_3_pos_std,hist_3_pos_amin,hist_3_pos_amax,post_position_mean,post_position_std,post_position_amin,post_position_amax,victory_count_mean,victory_count_std,victory_count_amin,victory_count_amax,placed_2_count_mean,placed_2_count_std,placed_2_count_amin,placed_2_count_amax,placed_3_count_mean,placed_3_count_std,placed_3_count_amin,placed_3_count_amax,victory_earnings_mean,victory_earnings_std,victory_earnings_amin,victory_earnings_amax,placed_earnings_mean,placed_earnings_std,placed_earnings_amin,placed_earnings_amax,prev_year_earnings_mean,prev_year_earnings_std,prev_year_earnings_amin,prev_year_earnings_amax,handicap_distance_mean,handicap_distance_std,handicap_distance_amin,handicap_distance_amax,handicap_weight_mean,handicap_weight_std,handicap_weight_amin,handicap_weight_amax,final_odds_ref_mean,final_odds_ref_std,final_odds_ref_amin,final_odds_ref_amax,final_odds_ref_offline_mean,final_odds_ref_offline_std,final_odds_ref_offline_amin,final_odds_ref_offline_amax,trueskill_mu_mean,trueskill_mu_std,trueskill_mu_amin,trueskill_mu_amax,trueskill_sigma_mean,trueskill_sigma_std,trueskill_sigma_amin,trueskill_sigma_amax,hist_1_days_mean,hist_1_days_std,hist_1_days_amin,hist_1_days_amax,hist_2_days_mean,hist_2_days_std,hist_2_days_amin,hist_2_days_amax,hist_3_days_mean,hist_3_days_std,hist_3_days_amin,hist_3_days_amax,trainer_winning_rate_mean,trainer_winning_rate_std,trainer_winning_rate_amin,trainer_winning_rate_amax,trainer_avg_winning_dividend_mean,trainer_avg_winning_dividend_std,trainer_avg_winning_dividend_amin,trainer_avg_winning_dividend_amax,race_count_r,hist_1_pos_r,hist_2_pos_r,hist_3_pos_r,post_position_r,victory_count_r,placed_2_count_r,placed_3_count_r,victory_earnings_r,placed_earnings_r,prev_year_earnings_r,handicap_distance_r,handicap_weight_r,final_odds_ref_r,final_odds_ref_offline_r,trueskill_mu_r,trueskill_sigma_r,hist_1_days_r,hist_2_days_r,hist_3_days_r,trainer_winning_rate_r,trainer_avg_winning_dividend_r,race_winner_dividend,odds_0,odds_1,odds_2,odds_3,odds_4,odds_5,odds_6,odds_7,odds_8,odds_9,odds_10,odds_11,odds_12,odds_13,odds_14,odds_15,odds_16,odds_17,odds_18,odds_19
count,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,1.413900e+04,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,11597.000000,14139.000000,14139.000000,14139.000000,14139.000000,5456.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,13945.000000,12518.000000,13541.000000,14139.000000,14139.000000,13517.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,10073.000000,14139.000000,14139.000000,1503.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.0000

In [6]:
filtered = df.groupby('race_id').filter(lambda r: (r['trueskill_mu'] == 25).sum() < r['declared_player_count'].max()/2)

In [7]:
filtered.describe()

,race_id,age,num,race_count,victory_count,placed_count,placed_2_count,placed_3_count,earnings,victory_earnings,placed_earnings,year_earnings,prev_year_earnings,post_position,position,handicap_weight,handicap_distance,trueskill_mu,trueskill_sigma,time,hist_1_days,hist_2_days,hist_3_days,winner_dividend,placed_dividend,final_odds,final_odds_offline,final_odds_unibet,final_odds_ref,final_odds_ref_offline,final_odds_ref_unibet,trainer_winning_rate,trainer_avg_winning_dividend,horse_id,trainer_id,jockey_id,herder_id,owner_id,num_race,num_bis,prize,distance,declared_player_count,session_id,num_session,hippodrome_id,hist_1_pos,hist_2_pos,hist_3_pos,hist_4_pos,hist_5_pos,hist_6_pos,speed,win,final_odds_ref_inv,race_count_mean,race_count_std,race_count_amin,race_count_amax,hist_1_pos_mean,hist_1_pos_std,hist_1_pos_amin,hist_1_pos_amax,hist_2_pos_mean,hist_2_pos_std,hist_2_pos_amin,hist_2_pos_amax,hist_3_pos_mean,hist_3_pos_std,hist_3_pos_amin,hist_3_pos_amax,post_position_mean,post_position_std,post_position_amin,post_position_amax,victory_count_mean,victory_count_std,victory_count_amin,victory_count_amax,placed_2_count_mean,placed_2_count_std,placed_2_count_amin,placed_2_count_amax,placed_3_count_mean,placed_3_count_std,placed_3_count_amin,placed_3_count_amax,victory_earnings_mean,victory_earnings_std,victory_earnings_amin,victory_earnings_amax,placed_earnings_mean,placed_earnings_std,placed_earnings_amin,placed_earnings_amax,prev_year_earnings_mean,prev_year_earnings_std,prev_year_earnings_amin,prev_year_earnings_amax,handicap_distance_mean,handicap_distance_std,handicap_distance_amin,handicap_distance_amax,handicap_weight_mean,handicap_weight_std,handicap_weight_amin,handicap_weight_amax,final_odds_ref_mean,final_odds_ref_std,final_odds_ref_amin,final_odds_ref_amax,final_odds_ref_offline_mean,final_odds_ref_offline_std,final_odds_ref_offline_amin,final_odds_ref_offline_amax,trueskill_mu_mean,trueskill_mu_std,trueskill_mu_amin,trueskill_mu_amax,trueskill_sigma_mean,trueskill_sigma_std,trueskill_sigma_amin,trueskill_sigma_amax,hist_1_days_mean,hist_1_days_std,hist_1_days_amin,hist_1_days_amax,hist_2_days_mean,hist_2_days_std,hist_2_days_amin,hist_2_days_amax,hist_3_days_mean,hist_3_days_std,hist_3_days_amin,hist_3_days_amax,trainer_winning_rate_mean,trainer_winning_rate_std,trainer_winning_rate_amin,trainer_winning_rate_amax,trainer_avg_winning_dividend_mean,trainer_avg_winning_dividend_std,trainer_avg_winning_dividend_amin,trainer_avg_winning_dividend_amax,race_count_r,hist_1_pos_r,hist_2_pos_r,hist_3_pos_r,post_position_r,victory_count_r,placed_2_count_r,placed_3_count_r,victory_earnings_r,placed_earnings_r,prev_year_earnings_r,handicap_distance_r,handicap_weight_r,final_odds_ref_r,final_odds_ref_offline_r,trueskill_mu_r,trueskill_sigma_r,hist_1_days_r,hist_2_days_r,hist_3_days_r,trainer_winning_rate_r,trainer_avg_winning_dividend_r,race_winner_dividend,odds_0,odds_1,odds_2,odds_3,odds_4,odds_5,odds_6,odds_7,odds_8,odds_9,odds_10,odds_11,odds_12,odds_13,odds_14,odds_15,odds_16,odds_17,odds_18,odds_19
count,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,1.224000e+04,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,9998.000000,12240.000000,12240.000000,12240.000000,12240.000000,5139.000000,12240.00000,12240.000000,12240.000000,12240.000000,12240.000000,12118.000000,11503.000000,11796.000000,12240.000000,12240.000000,11782.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,9616.000000,12240.000000,12240.000000,1503.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,

In [8]:
from cataclop.ml.exploration import random_race

In [20]:
random_race(filtered, ['start_at', 'category', 'sub_category', 'country', 'trainer_winning_rate', 'trainer_avg_winning_dividend', 'position', 'final_odds_ref', 'final_odds', 'trueskill_mu', 'trueskill_sigma'])

start_at category sub_category country  \
race_id id                                                         
80329   653481 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653482 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653483 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653484 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653485 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653486 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653487 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653488 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653489 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653490 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653491 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653492 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653493 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653494 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   
        653495 2020-01-12 14:40:00   ATTELE      INCONNU     FRA   

                trainer_winning_rate  trainer_avg_winning_dividend  position  \
race_id id                                                                     
80329   653481              0.090909                      1.181818      10.0   
        653482         100000.000000                 100000.000000       7.0   
        653483              0.038462                     -0.280769       4.0   
        653484              0.000000                     -1.000000       2.0   
        653485              0.000000                     -1.000000       NaN   
        653486              0.100000                     -0.600000       1.0   
        653487              0.250000                      2.025000       NaN   
        653488              0.000000                     -1.000000       3.0   
        653489              0.142857                      0.357143       8.0   
        653490              0.090909                     -0.736364       NaN   
        653491              0.352941                     -0.105882       NaN   
        653492              0.148148                     -0.322222       9.0   
        653493              0.000000                     -1.000000       6.0   
        653494              0.000000                     -1.000000       5.0   
        653495              0.000000                 100000.000000       NaN   

                final_odds_ref  final_odds  trueskill_mu  trueskill_sigma  
race_id id                                                                 
80329   653481           261.0       379.0     20.984982         1.452876  
        653482            29.0        46.0     23.796088         1.526003  
        653483            17.0        25.0     26.973391         1.554683  
        653484             5.0         4.6     27.651386         1.591680  
        653485            17.0        24.0     33.125674         1.692591  
        653486            10.0         7.8     23.695589         1.546596  
        653487             5.6         4.1     28.985058         1.640546  
        653488            16.0        17.0     30.730225         1.416016  
        653489           173.0       178.0     27.122153         1.364039  
        653490             8.8         9.7     25.487603         1.521063  
        653491             9.6        21.0     33.855955         1.612302  
        653492            50.0       150.0     20.500815         1.598145  
        653493            82.0       145.0     25.137053         2.106935  
        653494             8.5         5.8     28.231464         1.578693  
        653495             9.4        12.0     34.007687         1.527764

In [ ]:
from cataclop.core.models import Player, Horse, Race

p = Player.objects.get(id=649554)

print(p, p.music, p.race, p.trueskill_mu)

In [ ]:
hist = Player.objects.filter(horse=p.horse, imported_at__lt=p.imported_at).order_by('-imported_at')[0:10]
for h in hist:
    print(h, h.position, h.race)

In [2]:
from cataclop.core.enrich import compute_race_trueskill, compute_races_trueskill
race = Race.objects.get(id=80005)

In [ ]:
compute_race_trueskill(Race.objects.get(id=80005), force=True)

In [ ]:
for p in race.player_set.all():
    print(p, p.trueskill_mu, p.trueskill_sigma, p.position)

In [3]:
compute_races_trueskill(force=True)